In [ ]:
import os

os.chdir("..")

In [ ]:
import geopandas as gpd
from src.inserts import data_inserts 
import polars as pl
import requests
import sqlite3
import duckdb
di = data_inserts()

In [ ]:
conn = duckdb.connect()
conn.install_extension("sqlite")
conn.load_extension("sqlite")
conn.execute("ATTACH 'database.db' AS sqlite_db;")

In [ ]:
# Query the SQLite table using DuckDB
result = conn.execute("""
    SELECT * 
    FROM sqlite_db.year_table
    WHERE year = 2012;
""").fetchone()

if result:
    print("ID for year 2012:", result[0])
else:
    print("Year 2012 not found.")

# result

In [ ]:
# di.insert_regions()
# di.insert_states()
# di.insert_divisions()
# di.insert_county()
# di.insert_track()
# di.insert_urls()
# di.insert_years()

In [ ]:
di.data.select("c_geographyLink").to_series().to_list()

In [ ]:
def get_dataset_id(name):
    result = di.conn.execute(f"""
    SELECT * 
    FROM sqlite_db.urls_table
    WHERE api_url = '{name + "/"}';
""").fetchone()
    return result[0]

In [ ]:
get_dataset_id("abscb")

In [ ]:
base = "http://api.census.gov/data/2017/abscb/geography.json"
results = requests.get(base).json()
df = pl.DataFrame(results.get("fips"))
df

In [ ]:
def get_year_id(year):
    result = di.conn.execute(f"""
    SELECT * 
    FROM sqlite_db.year_table
    WHERE year = {year};
""").fetchone()
    return result[0]

In [ ]:
result = di.conn.execute(f"""
    SELECT * 
    FROM sqlite_db.year_table
    WHERE year = 2017;
""").fetchone()
type(result[0])

In [ ]:
yearx = base.split("/")[4]

get_year_id(yearx)

In [ ]:
for obs in df.select(pl.col("name") + "-" + pl.col("geoLevelDisplay")).to_series().to_list():
    geo_desc, geo_lv = obs.split("-")
    print()
    

In [ ]:
df.select("geoLevelDisplay").to_series().to_list()